In [ ]:
from IPython.display import HTML
# HTML(
#     """<script>code_show=true;function code_toggle()
# {if (code_show){$('div.input').hide();} 
# else{$('div.input').show();}code_show = !code_show}$( document ).ready(code_toggle);
# </script><em>The raw code in this jupyter notebook is hidden by default for easier reading.
# To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</,!a></em>."""
# )

In [ ]:
HTML("""
<style>

div.cell { /* Tunes the space between cells */
margin-top:5px;
margin-bottom:10px;
}

div.text_cell_render { /* Customize text cells */
font-family: 'Times New Roman';
font-size:20px;
line-height:1.4em;
padding-left:3em;
padding-right:3em;
}
</style>
""")

In [ ]:
# general modules
import numpy as np
import sys

import matplotlib.pyplot as plt
import subprocess
from pylfmap import LFmap

In [ ]:
# these modules are for crosschecks
from PyAstronomy import pyasl
import astropy
from astropy.coordinates import (
    SkyCoord,
    EarthLocation,
    AltAz,
    BaseCoordinateFrame,
    Galactic,
)
from astropy import units as u

# function for checking of the transformation rightness
def convertEquatorial2GalacticAndLocal(equatorialCoordinates, LST=None, altitude=None, **kwargs):
    ra, dec = equatorialCoordinates
    label = kwargs.get("label", None)
    print(label)
    gc = SkyCoord(ra=ra * u.degree, dec=dec * u.degree, frame="fk5")
    print(gc.galactic)
    print("RA:", ra, " ", "DEC:", dec)
    ha = LST * 15 - ra
    localCordinates = pyasl.hadec2altaz(ha, dec, -35.206667, ws=False, radian=False)
    print("ALT, AZ ", localCordinates)
    return gc.galactic, localCordinates

Import Polisensky's LFmap as a class  (you need to run the script LFmap_healpyFitsConvertorAndGenerator.py inside the LFmap software folder to generate and convert the fits format to healpy fits format).

In [ ]:
# healpy as the manipulation tool
import healpy as hp

# the upgraded newvisufunc is not yet in the official release
# current it is a pull request https://github.com/healpy/healpy/pull/648
from healpy.newvisufunc import projview

## Map loading
For the Polisensky's LFmaps you first have to convert the fits maps to healpy fits maps, to
run LFmap_healpyFitsConvertorAndGenerator.py in the LFmap folder.

In [ ]:
# for map from LFmap, these are by default generated in Celestial coordinated
# beware that pygdsm maps are on contrary by default in Galactic coordinates
g_LFmap = LFmap()

If you want to save the plots just add these line after the plot: <br>
plt.subplots_adjust(bottom=0.15,top=0.92) <br>
plt.savefig('./saveFolder/savedFile.png', facecolor='w', transparent=False) <br>
plt.close() <br>
Of course set the borders and file name as you like.

In [ ]:
cmap = "jet"

## Select frequency of interest

In [ ]:
map_LFmap = np.log(g_LFmap.generate(45.0))
map_LFmap_title = "LFmap"

## Galactic coordinate system

In [ ]:
# mollweide projections

projview(
    map_LFmap,
    coord=["G"],
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="l",
    ylabel="b",
    cb_orientation="vertical",
    min=8.4,
    max=11.5,
    latitude_grid_spacing=30,
    projection_type="mollweide",
    title=map_LFmap_title + " - Galactic coordinates",
    xtick_label_color="white",
    cmap=cmap,
)

## Equatorial coordinate system
Now the LFSS map has to be converted to celestioan (coord=['G','C']) and Polisensky's LFmap is fine.

In [ ]:
# Equatorial coordinates

projview(
    map_LFmap,
    coord=['G',"C"],
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="RA",
    ylabel="DEC",
    cb_orientation="vertical",
    min=8.4,
    max=11.5,
    latitude_grid_spacing=30,
    projection_type="mollweide",
    title=map_LFmap_title + " - Equatorial coordinates",
    xtick_label_color="white",
    cmap=cmap,
)

## Local coordinate system
<p style='text-align: justify;'>
The coordinate transformation is done from Equatorial coordinate system to local. So, make sure that both maps
are in Equatorial coordinate system when the rotation is perform. In the case of LFSS maps this means 
coord=['G','C'], and for Polisensky's maps just coord=['C']. The transformation to the local coordinate
system is done by Euler' rotation (ZYX). The rotation along the Z axis is given by the LST hourangle 
(LST hour times 15$^\circ$). The rotation along the Y is given by 90$^\circ$ + altitude of the local 
place.
   </p>

In [ ]:
LSTtime = 18
altitude = -35.206667

In [ ]:
# Local coordinates at LST time "LSTtime" at altitude "altitude"
rotAngles = [(180 + (LSTtime * 15)) % 360, -(altitude - 90)]

projview(
    map_LFmap,
    rot=rotAngles,
    coord=["G","C"],
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="azimuth",
    ylabel="altitude",
    cb_orientation="vertical",
    min=8.4,
    max=11.5,
    latitude_grid_spacing=30,
    projection_type="mollweide",
    title=map_LFmap_title + " - Local coordinates",
    xtick_label_color="white",
    cmap=cmap,
)

## Cartesian coordinate system (Galactic, Equatorial and local system)

In [ ]:
# Cartesian projections

projview(
    map_LFmap,
    coord=["G"],
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="l",
    ylabel="b",
    cb_orientation="vertical",
    min=8.4,
    max=11.5,
    latitude_grid_spacing=30,
    projection_type="cart",
    title=map_LFmap_title + " - Galactic coordinates",
    cmap=cmap,
)

In [ ]:
# Equatorial coordinates

projview(
    map_LFmap,
    coord=["G","C"],
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="RA",
    ylabel="DEC",
    cb_orientation="vertical",
    min=8.4,
    max=11.5,
    latitude_grid_spacing=30,
    projection_type="cart",
    title=map_LFmap_title + " - Equatorial coordinates",
    xtick_label_color="white",
    cmap=cmap,
)

In [ ]:
LSTtime = 18
# FYI PAO is at -35.2 altitude
altitude = -35.206667

In [ ]:
# Local coordinates at LST time "LSTtime" at altitude "altitude"
rotAngles = [(180 + (LSTtime * 15)) % 360, -(altitude - 90)]

projview(
    map_LFmap,
    rot=rotAngles,
    coord=["G","C"],
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="azimuth",
    ylabel="altitude",
    cb_orientation="vertical",
    min=8.4,
    max=11.5,
    latitude_grid_spacing=30,
    projection_type="cart",
    title=map_LFmap_title + " - Local coordinates",
    xtick_label_color="white",
    cmap=cmap,
)

## Cross-checks

<p style='text-align: justify;'>
The idea is to find point sources on the maps, calculate their coordinates in Galactic, Equatorial and Local
coordinate systems and check if the positions are the same on the plotted maps. (Yes, they are).
The values of the point sources are from Wiki, so they do not 100% correspond to the positions on the plots.
Check the original LFSS LF map reference paper 
https://www.mdpi.com/galaxies/galaxies-06-00056/article_deploy/html/images/galaxies-06-00056-g004.png
  </p>

In [ ]:
# cross checks with different tools
LSTtime=18

cygnusEquatorialCoordinates = (
    (19 + 58 / 60 + 21.67 / 3600) * 15,
    35 + 12 / 60 + 5.78 / 3600,
)
cygnusGalacticCoordinates, cygnusLocalCoordinates = convertEquatorial2GalacticAndLocal(
    cygnusEquatorialCoordinates, LST=LSTtime, altitude=altitude, label="Cygnus"
)

casiopeaEquatorialCoordinates = ((23 + 23 / 60 + 24 / 3600) * 15, 58 + 48.9 / 60)
(
    casiopeaGalacticCoordinates,
    casiopeaLocalCoordinates,
) = convertEquatorial2GalacticAndLocal(
    casiopeaEquatorialCoordinates, LST=LSTtime, altitude=altitude, label="Casiopea"
)

velaEquatorialCoordinates = ((9) * 15, -50)
velaGalacticCoordinates, velaLocalCoordinates = convertEquatorial2GalacticAndLocal(
    velaEquatorialCoordinates, LST=LSTtime, altitude=altitude, label="Vela"
)

centaurusEquatorialCoordinates = ((13 + 25 / 60 + 5 / 3600) * 15, -43 + 1 / 60)
(
    centaurusGalacticCoordinates,
    centaurusLocalCoordinates,
) = convertEquatorial2GalacticAndLocal(
    centaurusEquatorialCoordinates, LST=LSTtime, altitude=altitude, label="Centaurus A"
)

LMCEquatorialCoordinates = ((5 + +23 / 60 + 34 / 3600) * 15, -69 + 45 / 60 + 22 / 3600)
LMCGalacticCoordinates, LMCLocalCoordinates = convertEquatorial2GalacticAndLocal(
    LMCEquatorialCoordinates, LST=LSTtime, altitude=altitude, label="LMC"
)

CCEquatorialCoordinates = ((12 + 59/60 + 48.7/3600)*15, 27 + 58/60 + 50/3600)
CCGalacticCoordinates, CCLocalCoordinates = convertEquatorial2GalacticAndLocal(
    CCEquatorialCoordinates, LST=LSTtime, altitude=altitude, label="CC"
)

In [ ]:
def changeAzimuthCounterClockwiseConventionToSymmetrical(phi):
    if phi > 180:
        return 180 - phi % 180
    elif phi < 180:
        return -phi
    else:
        return phi


def changeAzimuthClockwiseConventionToSymmetrical(phi):
    if phi > 180:
        return phi%180-180
    elif ((phi > 0) & (phi <180)):
        return phi
    else:
        return phi


# mollweide projections
projview(
    map_LFmap,
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="l",
    ylabel="b",
    cb_orientation="vertical",
    min=8.4,
    max=11.5,
    latitude_grid_spacing=30,
    projection_type="mollweide",
    title=map_LFmap_title + " - Galactic coordinates",
    xtick_label_color="white",
    cmap=cmap,
)

# mollweide projections
projview(
    map_LFmap,
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="l",
    ylabel="b",
    cb_orientation="vertical",
    min=8.4,
    max=11.5,
    latitude_grid_spacing=30,
    projection_type="mollweide",
    title=map_LFmap_title + " - Galactic coordinates",
    xtick_label_color="white",
    cmap=cmap,
    phi_convention="symmetrical",
)


coordinateList = [cygnusGalacticCoordinates, casiopeaGalacticCoordinates, velaGalacticCoordinates,\
                 centaurusGalacticCoordinates, LMCGalacticCoordinates, CCGalacticCoordinates]

names = ["Cygnus", "Casiopea", "Vela", "Centaurus", "LMC", "Coma Cluster"]

for i, coordinate in enumerate(coordinateList):
    x = np.deg2rad(
        changeAzimuthCounterClockwiseConventionToSymmetrical(
            coordinate.l.value))
    y = np.deg2rad(coordinate.b.value)
    plt.scatter(x, y, color="r", marker="x", linewidth=2, s=70)
    plt.annotate(names[i], (x, y), color="red", fontsize=14)

In [ ]:
# mollweide projections
projview(
    map_LFmap,
    coord=["G","C"],
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="l",
    ylabel="b",
    cb_orientation="vertical",
    min=8.4,
    max=11.5,
    latitude_grid_spacing=30,
    projection_type="mollweide",
    title=map_LFmap_title + " - Equatorial coordinates",
    xtick_label_color="white",
    cmap=cmap,
    phi_convention="symmetrical",
)

coordinateList = [
    cygnusEquatorialCoordinates,
    casiopeaEquatorialCoordinates,
    velaEquatorialCoordinates,
    centaurusEquatorialCoordinates,
    LMCEquatorialCoordinates,
    CCEquatorialCoordinates,
]


names = ["Cygnus", "Casiopea", "Vela", "Centaurus", "LMC", "Coma Cluster"]

for i, coordinate in enumerate(coordinateList):
    x, y = coordinate
    x = np.deg2rad(changeAzimuthCounterClockwiseConventionToSymmetrical(x))
    y = np.deg2rad(y)
    plt.scatter(x, y, color="r", marker="x", linewidth=2, s=70)
    plt.annotate(names[i], (x, y), color="red", fontsize=14)

In [ ]:
# Local coordinates at LST time "LSTtime" at altitude "altitude"
rotAngles = [(180 + (LSTtime * 15)) % 360, -(altitude - 90)]
projview(
    map_LFmap,
    rot=rotAngles,
    coord=["G","C"],
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="azimuth",
    ylabel="altitude",
    cb_orientation="vertical",
    min=8.4,
    max=11.5,
    latitude_grid_spacing=30,
    projection_type="mollweide",
    title=map_LFmap_title + " - Local coordinates",
    xtick_label_color="white",
    cmap=cmap,
    phi_convention="symmetrical",
)

projview(
    map_LFmap,
    rot=rotAngles,
    coord=["G","C"],
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="azimuth",
    ylabel="altitude",
    cb_orientation="vertical",
    min=8.4,
    max=11.5,
    latitude_grid_spacing=30,
    projection_type="mollweide",
    title=map_LFmap_title + " - Local coordinates",
    xtick_label_color="white",
    cmap=cmap,
    phi_convention="clockwise",
)

x = [
    np.deg2rad(changeAzimuthClockwiseConventionToSymmetrical(cygnusLocalCoordinates[1])),
    np.deg2rad(
        changeAzimuthClockwiseConventionToSymmetrical(casiopeaLocalCoordinates[1])
    ),
    np.deg2rad(
        changeAzimuthClockwiseConventionToSymmetrical(centaurusLocalCoordinates[1])
    ),
    np.deg2rad(changeAzimuthClockwiseConventionToSymmetrical(LMCLocalCoordinates[1])),
]

y = [
    np.deg2rad(cygnusLocalCoordinates[0]),
    np.deg2rad(casiopeaLocalCoordinates[0]),
    np.deg2rad(centaurusLocalCoordinates[0]),
    np.deg2rad(LMCLocalCoordinates[0]),
]

coordinateList = [
    cygnusLocalCoordinates,
    casiopeaLocalCoordinates,
    velaLocalCoordinates,
    centaurusLocalCoordinates,
    LMCLocalCoordinates,
    CCLocalCoordinates,
]


names = ["Cygnus", "Casiopea", "Vela", "Centaurus", "LMC", "Coma Cluster"]

for i, coordinate in enumerate(coordinateList):
    y, x = coordinate
    x = np.deg2rad(changeAzimuthClockwiseConventionToSymmetrical(x))
    y = np.deg2rad(y)
    plt.scatter(x, y, color="r", marker="x", linewidth=2, s=70)
    plt.annotate(names[i], (x, y), color='red', fontsize=14)

## Some other projection types
Just a demonstration of what can healpy plot.
projection_type :  {'aitoff', 'hammer', 'lambert', 'mollweide', 'cart', '3d', 'polar'}

In [ ]:
# 3d projection, vertical cbar
projview(
    map_LFmap,
    hold=False,
    graticule=True,
    graticule_labels=True,
    projection_type="3d",
    unit="cbar label",
    xlabel="xlabel",
    ylabel="ylabel",
    cb_orientation="vertical",
    cmap=cmap,
)

In [ ]:
# polar projection, vertical cbar
projview(
    map_LFmap,
    graticule=True,
    graticule_labels=True,
    unit="cbar label",
    cb_orientation="vertical",
    projection_type="polar",
    xtick_label_color="white",
    cmap=cmap,
)

In [ ]:
# hammer projection, vertical cbar
projview(
    map_LFmap,
    graticule=True,
    graticule_labels=True,
    unit="cbar label",
    cb_orientation="vertical",
    projection_type="hammer",
    xtick_label_color="white",
    cmap=cmap,
)

## Data dump
This is very usefull.

In [ ]:
# return only data
# [longitude,  latitude, grid_map]
# longitude,  latitude are 1D arrays to convert them to 2D arrays for the plot use np.meshgrid(longitude,  latitude)
# longtitude goes from -pi to pi (-180 to 180 in degs)
# latitude goes from -pi/2 to pi/2 (-90 to 90 in degs)
longitude, latitude, grid_map = projview(map_LFmap, coord=["C", "G"], return_only_data=True, xsize=3600)
print(longitude.shape)
print(latitude.shape)
print(grid_map.shape)